# **필요한 라이브러리 불러오기**

In [1]:
import pandas as pd
import joblib
import seaborn as sns
from tqdm import tqdm

# **데이터, 모델, 스케일러 불러오기**

In [2]:
transactions = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/머신러닝_ZBDS/modeling/sample_data (ratio = 0.5)/sample_transactions (ratio = 0.5).parquet')
data_target = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/머신러닝_ZBDS/modeling/data_target.parquet')
model = joblib.load('/content/drive/MyDrive/Colab Notebooks/머신러닝_ZBDS/modeling/churn_predict_best_lgbm.pkl')
ct = joblib.load('/content/drive/MyDrive/Colab Notebooks/머신러닝_ZBDS/modeling/column_transformer.pkl')

# **이탈 확률 구간별 LTV 계산**

* ARPU는 아래와 같이 계산되었습니다.
  * 본 데이터셋의 구매 기록은 2015년 1월 1일부터 2017년 3월 31일사이를 대상으로 함 (27개월)

$$총\,매출 = 대상\,기간\,동안의\,그룹별\,매출$$
$$ARPU = \frac{총\,매출 / 27개월}{그룹별\,전체\,고객\,수}$$

* LTV는 아래와 같이 계산되었습니다.

$$LTV = \frac{ARPU}{그룹별\,실제\,이탈유저의\,비율}$$


In [3]:
# 각 유저의 이탈 확률 (predict_proba) 구하기
input = pd.DataFrame(ct.transform(data_target.iloc[:, 1:-1]), columns = ct.get_feature_names_out())
data_target['proba'] = model.predict_proba(input)[:, 1].tolist()
data_target['proba'] = data_target['proba']*100

# 이탈 확률의 값에 따라 전체 유저를 Grouping하기
for row_num in tqdm(range(data_target.shape[0])):
  for thres in [[i*10, (i+1)*10, str(i*10) + ' ~ ' + str((i+1)*10)] for i in range(10)]:
    con_1 = data_target.loc[row_num, 'proba'] >= thres[0]
    con_2 = data_target.loc[row_num, 'proba'] < thres[1]
    if con_1 & con_2:
      data_target.loc[row_num, 'thres'] = thres[2]
      break

# 구매기록에 기초하여 총 매출을 계산
total_revenue = pd.DataFrame(transactions.groupby('msno_num', as_index = False)['actual_amount_paid'].sum())
total_revenue.rename({'actual_amount_paid' : '전체 매출(NTD)'}, axis = 1, inplace = True)
total_revenue['전체 매출(NTD)'] = total_revenue['전체 매출(NTD)'] / 27
data_target = data_target.merge(total_revenue, on = 'msno_num', how = 'inner')

# 그룹별로 잔존 유저와 이탈 유저, 전체 유저수, 이탈 유저의 비율을 계산
segmentation = pd.pivot_table(data_target, index = ['thres'], columns = ['is_churn'], values = ['msno_num'], aggfunc = 'count')
segmentation.columns = segmentation.columns.droplevel(0)
segmentation.rename({0 : '잔존 유저(명)', 1 : '이탈 유저(명)'}, axis = 1, inplace = True)
segmentation['전체 유저수(명)'] = segmentation['잔존 유저(명)'] + segmentation['이탈 유저(명)']
segmentation['이탈유저의 비율(%)'] = round(segmentation['이탈 유저(명)'] / segmentation['전체 유저수(명)'], 4)
segmentation.reset_index(inplace = True)
segmentation.columns.name = ''

# 그룹별 총 매출을 병합
total_revenue = pd.DataFrame(data_target.groupby('thres', as_index = False)['전체 매출(NTD)'].sum())
segmentation = segmentation.merge(total_revenue, on = 'thres', how = 'inner')
segmentation.rename({'thres' : '이탈 확률(%)'}, axis = 1, inplace = True)

# ARPU, LTV를 계산
segmentation['ARPU(NTD)'] = round(segmentation['전체 매출(NTD)'] / segmentation['전체 유저수(명)'])
segmentation['LTV(NTD)'] = round(segmentation['ARPU(NTD)'] / segmentation['이탈유저의 비율(%)'])
segmentation['이탈유저의 비율(%)'] = segmentation['이탈유저의 비율(%)']*100

segmentation.drop('전체 매출(NTD)', axis = 1, inplace = True)
segmentation

100%|██████████| 476471/476471 [01:39<00:00, 4803.03it/s]


,이탈 확률(%),잔존 유저(명),이탈 유저(명),전체 유저수(명),이탈유저의 비율(%),ARPU(NTD),LTV(NTD)
0,0 ~ 10,334485,3399,337884,1.01,88.0,8713.0
1,10 ~ 20,23580,3602,27182,13.25,111.0,838.0
2,20 ~ 30,23136,6471,29607,21.86,90.0,412.0
3,30 ~ 40,16500,8522,25022,34.06,74.0,217.0
4,40 ~ 50,7968,7609,15577,48.85,71.0,145.0
5,50 ~ 60,4278,6631,10909,60.78,83.0,137.0
6,60 ~ 70,2867,7173,10040,71.44,87.0,122.0
7,70 ~ 80,1406,7095,8501,83.46,88.0,105.0
8,80 ~ 90,563,6460,7023,91.98,51.0,55.0
9,90 ~ 100,169,4557,4726,96.42,61.0,63.0
